<a href="https://colab.research.google.com/github/hyunBBB/OMG/blob/main/06_%EB%B3%B4%EC%8A%A4%ED%84%B4_%EC%A7%91%EA%B0%92_%EC%98%88%EC%B8%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
"""## DNN(MLP) 모델을 이용한 보스턴 집값 데이터 셋 회귀 분석"""

import tensorflow as tf
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
# 이걸 스텝원에다가 하는거임 아래에다가

In [8]:
"""* Step 1. Inptu tensor 와 Target tensor 준비(훈련데이터)"""

# TODO
# 1. import 한 boston_housing API를 이용하여 boston_housing 데이터 셋을 다운로드
boston_housing_url = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/boston_housing.npz'
dataset_path = tf.keras.utils.get_file("/content/boston_housing.data", boston_housing_url)
dataset_path
# 이건 안ㅆ더 되나봄


'/content/boston_housing.data'

In [22]:

"""* Step 2. 데이터의 전처리 """

# TODO
# mean = train_data.mean(), std = train_data.std()
# 1. train 데이터의 feature 별 평균값, 표준편차 값을 이용하여 정규화 작업을 진행 각 엑스값에서 평균값빼고 그고 하면 됨.
#  정규화는 z = (x-mean)/std

mean = train_data.mean(axis=0)
train_data -= mean  
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std




In [24]:
train_data.shape

(404, 13)

In [12]:

"""* Step 3. DNN(MLP) 모델 디자인"""

# TODO
# 1. Sequential API를 이용하여 boston_housing 데이터 셋을 분석 하기 위한 MLP 모델을 디자인 seq열어서 에드에드 하면되고.
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [25]:
# 이게 쌤이 하신거임

from tensorflow.keras import models, layers
model = models.Sequential()

model.add(layers.Dense(64, activation = 'relu', input_shape=(train_data.shape[1], )))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(1)) # 집값하나만 하려고하는거니까 1이 들어가야함.

In [26]:
model.summary()
# model형태를 볼 수 있음. 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                896       
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [15]:

"""* Step 4. 모델의 학습 정보 설정"""

# TODO
# 1. tf.keras.Model 객체의 compile 메서드를 이용하여 학습을 위한 정보들을 설정
#   - optimizer
#   - loss
#   - metrics : 체점 기준인 MAE 로 설정

import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
all_mae_histores = []
for i in range(k):
    print('processing fold #', i)
    # 검증 데이터 분리
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    # 훈련 데이터 분리
    partial_train_data = np.concatenate([train_data[:i*num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i*num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)



processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [27]:
# 쌤이 하신거
model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])

In [19]:

"""* Step 5. 모델에 input, target 데이터 연결 후 학습"""

# TODO
# 1. tf.keras.Model 객체의 fit 메서드를 이용하여 모델을 학습하세요
#   - fit 메서드의 verbose=2 로 설정 하세요

model = build_model()
history = model.fit(partial_train_data,
                    partial_train_targets, validation_data=(val_data, val_targets), 
                   epochs=num_epochs, batch_size=1, verbose=2)

mae_history = history.history['val_mean_absolute_error']
all_mae_histories.append(mae_history)

val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 2)
all_scores.append(val_mae)


Epoch 1/100
303/303 - 1s - loss: 391.2737 - mae: 17.7371 - val_loss: 343.3365 - val_mae: 15.7443 - 975ms/epoch - 3ms/step
Epoch 2/100
303/303 - 1s - loss: 128.3422 - mae: 8.6765 - val_loss: 123.9350 - val_mae: 7.3319 - 515ms/epoch - 2ms/step
Epoch 3/100
303/303 - 1s - loss: 50.7650 - mae: 4.7734 - val_loss: 89.4310 - val_mae: 6.0765 - 515ms/epoch - 2ms/step
Epoch 4/100
303/303 - 1s - loss: 37.5266 - mae: 4.0209 - val_loss: 70.0984 - val_mae: 5.3413 - 507ms/epoch - 2ms/step
Epoch 5/100
303/303 - 0s - loss: 30.5634 - mae: 3.5804 - val_loss: 59.5455 - val_mae: 4.8254 - 493ms/epoch - 2ms/step
Epoch 6/100
303/303 - 0s - loss: 26.6664 - mae: 3.2783 - val_loss: 53.2153 - val_mae: 4.4410 - 466ms/epoch - 2ms/step
Epoch 7/100
303/303 - 1s - loss: 24.0396 - mae: 3.0863 - val_loss: 46.6355 - val_mae: 4.1980 - 513ms/epoch - 2ms/step
Epoch 8/100
303/303 - 1s - loss: 22.3032 - mae: 2.9770 - val_loss: 43.7605 - val_mae: 3.9739 - 505ms/epoch - 2ms/step
Epoch 9/100
303/303 - 0s - loss: 20.5778 - mae: 2.

KeyError: ignored

In [31]:
# 쌤이 하신거
history = model.fit( x = train_data, y = train_labels,
                    epochs = 100, batch_size = 1,
                    validation_split = 0.2)

NameError: ignored

In [32]:

"""* 모델 제출 """

# 학습된 모델을 제출하기 위한 코드 입니다. 수정하지 마세요
model.save('my_model.h5')